In [8]:
# 체험
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv

chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')

driver = webdriver.Chrome(chromedriver, options=options)

# areacode 조정해서 지역 선택
driver.get("https://korean.visitkorea.or.kr/list/ms_list.do?areacode=31")

print(driver.title)
print(driver.current_url)

# nth-child 숫자 조정해서 태그 선택
tag = driver.find_element_by_css_selector("#taglist> li:nth-child(8) > button > span")

actions = webdriver.ActionChains(driver)
actions.move_to_element(tag)
actions.click(tag)
actions.perform()
    
time.sleep(5)

# 인기순
rank = driver.find_element_by_css_selector(".btn_txt > button[id='3']")
actions = webdriver.ActionChains(driver)
actions.move_to_element(rank)
actions.double_click(rank)
actions.perform()
    
time.sleep(5)

# 데이터가 1000개 넘는 경우 대비
if ',' in driver.find_element_by_css_selector("#totalCnt").text:
    data_num = int(''.join(driver.find_element_by_css_selector("#totalCnt").text.split(',')))
else:
    data_num = int(driver.find_element_by_css_selector("#totalCnt").text)
    
print(data_num)

# csv 파일 이름에 지역 넣기 위해 선언
region = driver.find_element_by_css_selector(f"#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child(1) > div.area_txt > p:nth-child(2)") 

# 지역에 현재 위치 반영될 경우 대비
if "-" in region.text:
    region = region.text.split("-")[1]
    region = region[1:]
else:
    region = region.text
    
travel_data = []

# csv 파일은 크롤링 시작 전 오픈(중간에 끊어질 경우 대비하여 데이터 하나 크롤링 끝나면 csv 파일에 데이터 추가)
with open(f'./travel_data_experience/{region[:2]}_experience_data_address.csv', 'a', encoding='utf-8-sig', newline='') as file:
    fieldnames = ['name', 'region', 'address']
    csvfile = csv.DictWriter(file, fieldnames = fieldnames)
    csvfile.writeheader()
    
    # 크롤링 끊어졌을 경우 크롤링을 다시 시작해야 하는 위치로 이동
    for k in range(0, 6, 50):
        page = driver.find_element_by_css_selector(f".page_box > a[id='{(k // 10) + 1}']")
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(page)
        actions.double_click(page)
        actions.perform()

        time.sleep(5)
    
    # 크롤링 반복문
    for j in range(6, data_num):
        
        # 페이지 이동
        if ((j // 10) + 1) > 1:
            page = driver.find_element_by_css_selector(f".page_box > a[id='{(j // 10) + 1}']")
            actions = webdriver.ActionChains(driver)
            actions.move_to_element(page)
            actions.double_click(page)
            actions.perform()

            time.sleep(5)

        name = driver.find_element_by_css_selector(f"#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({(j % 10) + 1}) > div.area_txt > div > a")
        name_text = name.text
        region = driver.find_element_by_css_selector(f"#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({(j % 10) + 1}) > div.area_txt > p:nth-child(2)") 
        
        # 지역에 현재 위치 반영될 경우 대비
        if "-" in region.text:
            region = region.text.split("-")[1]
            region = region[1:]
        else:
            region = region.text
        
        # 주소를 수집하기 위해 여행지 상세정보 페이지로 이동
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(name)
        actions.double_click(name)
        actions.perform()
        
        time.sleep(5)
        
        # 주소 수집(예외 처리를 위한 조건문)
        if len(driver.find_elements_by_css_selector("#detailGo > div:nth-child(3) > div > div.inr_wrap > div > ul > li")) >= 3:
            address = driver.find_element_by_css_selector("#detailGo > div:nth-child(3) > div > div.inr_wrap > div > ul > li:nth-child(3) > span").text
        elif len(driver.find_elements_by_css_selector("#detailGo > div:nth-child(3) > div > div.inr_wrap > div > ul > li")) == 2:
            address = driver.find_element_by_css_selector("#detailGo > div:nth-child(3) > div > div.inr_wrap > div > ul > li:nth-child(2) > span").text
        else:
            address = driver.find_element_by_css_selector("#detailGo > div:nth-child(3) > div > div.inr_wrap > div > ul > li > span").text

        travel_dict = {
            'name' : name_text,
            'region' : region,
            'address' : address
        }

        # csv 파일에 데이터 추가
        travel_data.append(travel_dict)
        csvfile.writerow(travel_dict)
        
        # 뒤로 이동하여 다음 데이터 크롤링 진행
        back = driver.find_element_by_css_selector("#btnPrev")
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(driver.find_element_by_css_selector("#contents > div.titleType1 > div.area_tag > span > a"))
        actions.move_to_element(back)
        actions.click(back)
        actions.perform()

        time.sleep(5)



대한민국구석구석 | 여행지 목록
https://korean.visitkorea.or.kr/list/ms_list.do?areacode=31
159
